In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
#Y110.shp
# if DIR=-1 or 2, add a link from B to A
# finally if DIR=-1, eliminate the link

gdf = gpd.read_file("Y110.shp")

for i, row in gdf.iterrows():
    if row['DIR'] == -1 or row['DIR'] == 2:
        # Create a new row with the A and B coordinates reversed
        new_row = row.copy()
        new_row['A'] = row['B']
        new_row['B'] = row['A']
        if row['DIR'] == -1:
            new_row['DIR'] = 1
        else:
            new_row['DIR'] = 2

        # Append the new row to the GeoDataFrame
        gdf = gdf.append(new_row, ignore_index=True)

# Eliminate the rows where DIR = -1
gdf = gdf[gdf['DIR'] != -1]

# Save the modified GeoDataFrame to a new shapefile
gdf.to_file("Y110_modified.shp", driver='ESRI Shapefile')

In [ ]:
ue_results = pd.read_csv("UE_results.dat", sep="\t")
gdf = gpd.read_file("Y110_modified_V1.shp")

# Rename columns to match
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

# Merge results back into the shapefile
gdf = gdf.merge(ue_results[["A", "B", "capacity", "UE_travelTime", "UE_flow"]], on=["A", "B"], how="left")

# Calculate V/C ratio
gdf["V/C"] = gdf["UE_flow"] / gdf["capacity"]

# Save updated shapefile
gdf.to_file("UE_results.shp")